# Subtitle Collection

## Step #1: Collect Youtube Video ID list for Children's works

- The Youtube API now has a madeforkids flag
   - https://developers.google.com/youtube/v3/docs/videos
- VideoCategories can be listed at https://developers.google.com/youtube/v3/docs/videoCategories/list?apix_params=%7B%22part%22%3A%5B%22snippet%22%5D%2C%22regionCode%22%3A%22us%22%7D (part:'snippet' ensures that the name is included)




In [82]:
%load_ext autoreload
%autoreload 2

In [558]:
import pandas as pd
import json
import os
import math
import glob
import numpy as np
import dask.dataframe as dd
import youtube as yt #youtube.py
import youtube_transcript_api

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../client_secret_312872897581-qii8av1f39ko8iavk4qp3sm8lqnp74op.apps.googleusercontent.com.json"

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)

credentials = flow.run_console()

youtube_api = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [297]:
q = "children|kids|kid|teen|teens -voice -music -spirit"
topicId = '/m/01k8wb' # Educational; see: https://developers.google.com/youtube/v3/docs/search/list
#topicId = '/m/02jjt' # Entertainment 
#topicId = '/m/0f2f9' # TV-shows - a subset of education
#topicId = None
# topicId = '/m/09kqc' # Humor
#topicId = '/m/0bzvm2' # Gaming
sortBy = 'rating' # date | rating | relevance | title | videoCount | viewCount

quota_units_to_use = 2000
quota_cost = 100
n_calls = int(quota_units_to_use / quota_cost)

https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=motes-283322&pageState=(%22duration%22:(%22groupValue%22:%22PT1H%22))

https://developers.google.com/youtube/v3/docs/search/list

In [449]:
for topicId in ['/m/0f2f9','/m/0403l3g']: #,'/m/025zzc']: #['/m/019_rr', '/m/02vxn', '/m/01k8wb', '/m/02jjt', '/m/09kqc', None, '/m/03glg', '/m/0bzvm2']:
    for sortBy in ['date']: #, 'relevance', 'viewCount', 'rating']:
        pt = None
        results_collector = []

        for i in range(20):
            pt, df = yt.search_youtube(youtube_api, q, pageToken=pt, topicId=topicId, order=sortBy)
            print(i, 'Next pageToken:', pt)
            results_collector.append(df)
            if (pt is None) or (df.shape[0] == 0):
                break

        all_searches = pd.concat(results_collector)
        if all_searches.empty:
            continue
        all_searches['q'] = q
        all_searches['topicId'] = topicId
        all_searches['sortBy'] = sortBy
        now = int(time.time())
        all_searches.to_parquet('data/yt_info/initial_search_results_{}.parquet'.format(now))

0 Next pageToken: CDIQAA
1 Next pageToken: CGQQAA
2 Next pageToken: CJYBEAA
3 Next pageToken: CMgBEAA
4 Next pageToken: CPoBEAA
5 Next pageToken: CKwCEAA
6 Next pageToken: CN4CEAA
7 Next pageToken: CJADEAA
8 Next pageToken: CMIDEAA
9 Next pageToken: CPQDEAA
10 Next pageToken: CKYEEAA
0 Next pageToken: CDIQAA
1 Next pageToken: CGQQAA
2 Next pageToken: CJYBEAA
3 Next pageToken: CMgBEAA
4 Next pageToken: CPoBEAA
5 Next pageToken: CKwCEAA
6 Next pageToken: CN4CEAA
7 Next pageToken: CJADEAA


### Download Details

Particularly important is whether the video is 'madeForKids'.

In [ ]:
for sortBy in ['date', 'relevance', 'viewCount', 'rating']:
    pt = None
    results_collector = []

    for i in range(10):
        pt, df = yt.search_youtube(youtube_api, q, pageToken=pt, topicId=topicId, order=sortBy)
        print(i, 'Next pageToken:', pt)
        results_collector.append(df)
        if (pt is None) or (df.shape[0] == 0):
            break

    all_searches = pd.concat(results_collector)
    if all_searches.empty:
        continue
    all_searches['q'] = q
    all_searches['topicId'] = topicId
    all_searches['sortBy'] = sortBy
    now = int(time.time())
    all_searches.to_parquet('data/yt_info/initial_search_results_{}.parquet'.format(now))

In [300]:
# See which conditions have been collected
df = dd.read_parquet('data/yt_info/initial_search_results_*').compute()
df[['q', 'topicId', 'sortBy']].drop_duplicates()

,q,topicId,sortBy
0,children|kids|kid|teen|teens,01k8wb,relevance
0,children|kids|kid|teen|teens,02jjt,relevance
0,children|kids|kid|teen|teens -voice -music -sp...,02jjt,relevance
0,children|kids|kid|teen|teens -voice -music -sp...,/m/02jjt,viewCount
0,children|kids|kid|teen|teens -voice -music -sp...,/m/02jjt,rating
0,children|kids|kid|teen|teens -voice -music -sp...,/m/01k8wb,viewCount
0,children|kids|kid|teen|teens -voice -music -sp...,/m/01k8wb,rating
0,children|kids|kid|teen|teens -voice -music -sp...,/m/0bzvm2,date
0,children|kids|kid|teen|teens -voice -music -sp...,/m/0bzvm2,relevance
0,children|kids|kid|teen|teens -voice -music -sp...,/m/0bzvm2,viewCount


TODO add details by file

In [452]:
# Find the first file of *initial* search results that hasn't been queried for
# extra info yet.
data_paths = glob.glob('data/yt_info/*')
for initial_path in [x for x in data_paths if 'initial' in x]:
    details_path = initial_path.replace('initial', 'details')
    if details_path in data_paths:
        print(initial_path, details_path)
        continue
    else:
        df = pd.read_parquet(initial_path).drop_duplicates('videoId')
        df2 = yt.augment_initial_search(youtube_api, df)
        df2.to_parquet(details_path)

data/initial_search_results_1598497187.parquet data/details_search_results_1598497187.parquet
0,1,2,3,4,5,6,7,8,9,10,11,
data/initial_search_results_1598719429.parquet data/details_search_results_1598719429.parquet
data/initial_search_results_1598718866.parquet data/details_search_results_1598718866.parquet
data/initial_search_results_1598720863.parquet data/details_search_results_1598720863.parquet
data/initial_search_results_1598721181.parquet data/details_search_results_1598721181.parquet
data/initial_search_results_1598721188.parquet data/details_search_results_1598721188.parquet
data/initial_search_results_1598839262.parquet data/details_search_results_1598839262.parquet
data/initial_search_results_1598839272.parquet data/details_search_results_1598839272.parquet
data/initial_search_results_1598839280.parquet data/details_search_results_1598839280.parquet
data/initial_search_results_1598839287.parquet data/details_search_results_1598839287.parquet
0,1,2,3,4,5,6,7,8,9,10,
0,1,2,3,4

### Investigate common words in made for kids titles vs others

In [497]:
df = dd.read_parquet(glob.glob('data/yt_info/details_*')).drop_duplicates(['videoId']).compute()

In [567]:
df

,videoId,title,description,publishedAt,channelTitle,q,topicId,sortBy,duration,caption,relevantTopicIds,topicCategories,madeForKids
0,89eCHtFs0XM,Internet Safety for Kids K-3,Internet safety for kids is a topic getting mu...,2011-05-17T18:05:28Z,Indiana University of Pennsylvania,children|kids|kid|teen|teens,01k8wb,relevance,PT7M51S,true,"[/m/01k8wb, /m/01k8wb]",[https://en.wikipedia.org/wiki/Knowledge],0.0
1,Uo8lbeVVb4M,Be a Weather Watcher | Science for Kids,Learn about different types of weather and how...,2016-04-05T20:08:39Z,SciShow Kids,children|kids|kid|teen|teens,01k8wb,relevance,PT3M59S,true,"[/m/01k8wb, /m/01k8wb]",[https://en.wikipedia.org/wiki/Knowledge],1.0
2,EwY6p-r_hyU,Gravity Compilation: Crash Course Kids,Maybe you'd like to just hear about one topic ...,2016-06-10T20:30:01Z,Crash Course Kids,children|kids|kid|teen|teens,01k8wb,relevance,PT14M33S,true,"[/m/01k8wb, /m/01k8wb]",[https://en.wikipedia.org/wiki/Knowledge],1.0
3,nx3GuO41Jyg,Cameron Herold: Let&#39;s raise kids to be ent...,"http://www.ted.com Bored in school, failing cl...",2010-06-17T16:41:14Z,TED,children|kids|kid|teen|teens,01k8wb,relevance,PT22M9S,true,"[/m/01k8wb, /m/01k8wb]",[https://en.wikipedia.org/wiki/Knowledge],0.0
4,z9TIlM96lT8,Gotta Eat! - Crash Course Kids 1.1,Welcome to Crash Course Kids. In this first ep...,2015-03-03T22:30:01Z,Crash Course Kids,children|kids|kid|teen|teens,01k8wb,relevance,PT2M56S,true,"[/m/01k8wb, /m/01k8wb]",[https://en.wikipedia.org/wiki/Knowledge],1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,895REvV276U,How I Made I Little Kid Cry- Mariocow,I'm Such a Jerk.,2013-01-25T22:23:18Z,TheMinecraftDopes,children|kids|kid|teen|teens -voice -music -sp...,/m/0403l3g,date,PT31S,true,"[/m/0bzvm2, /m/025zzc, /m/0403l3g, /m/0bzvm2, ...",[https://en.wikipedia.org/wiki/Video_game_cult...,0.0
333,MDg_prV0_TU,SoulSplit some kid scamming me,idiot kid scamming me pls report him if u can ...,2012-11-02T07:39:59Z,Victor Plugovoy,children|kids|kid|teen|teens -voice -music -sp...,/m/0403l3g,date,PT2M28S,true,"[/m/0bzvm2, /m/0403l3g, /m/0bzvm2, /m/0403l3g]",[https://en.wikipedia.org/wiki/Video_game_cult...,0.0
334,K6ZhVK4h2Ps,Raiding a Super RICH OP Kid in Minecraft,We go on an insane raid and steal from a super...,2012-09-25T23:26:36Z,housewars1,children|kids|kid|teen|teens -voice -music -sp...,/m/0403l3g,date,PT14M36S,true,"[/m/0403l3g, /m/0bzvm2, /m/025zzc, /m/0bzvm2, ...",[https://en.wikipedia.org/wiki/Video_game_cult...,0.0
335,juHKlQujdBg,Hexana.net war Children,Children vs Opposition.,2012-02-13T22:21:58Z,Nikoś SucheMajtki,children|kids|kid|teen|teens -voice -music -sp...,/m/0403l3g,date,PT1M59S,true,"[/m/0bzvm2, /m/0403l3g, /m/0bzvm2, /m/0403l3g]",[https://en.wikipedia.org/wiki/Video_game_cult...,0.0


What are the top topics ids?

In [451]:
# Top topic ids. obviously skewed to the ones searched for!
past_searches= ['/m/0f2f9', '/m/0403l3g', '/m/019_rr', '/m/02vxn', '/m/01k8wb', '/m/02jjt', '/m/09kqc', None, '/m/03glg', '/m/0bzvm2']
kids = df[df.madeForKids > 0]
pd.Series([x for y in kids.relevantTopicIds.dropna().tolist() for x in y if x in past_searches]).value_counts()

/m/02jjt      1082
/m/02vxn       828
/m/01k8wb      502
/m/0bzvm2      372
/m/0f2f9       201
/m/0403l3g      35
/m/09kqc        20
dtype: int64

### What are the top words used in kids titles vs non-madeForKids titles?

Using odds-ratio here, without a correction for missing words.

In [428]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [498]:
kids_titles = " ".join(df[df.madeForKids > 0].title.tolist())
adult_titles = " ".join(df[df.madeForKids == 0].title.tolist())

kids_words = pd.Series([word.lemma_.lower() for word in nlp(kids_titles)]).value_counts()
adult_words = pd.Series([word.lemma_.lower() for word in nlp(adult_titles)]).value_counts()

kodds = kids_words / kids_words.sum()
aodds = adult_words / adult_words.sum()
odds_ratio = (kodds/aodds).dropna()

In [535]:
odds_ratio.sort_values(ascending=False).head(20)

squad          45.259750
alphabet       45.259750
wild           32.494179
education      30.007380
odd            29.012660
science        27.852154
handwriting    25.531141
episodes       23.210128
animated       22.049622
cbc            20.889115
wow            20.889115
vocabulary     17.407596
planet         16.247090
bruder         16.247090
coloring       15.086583
colors         13.345824
solar          12.765570
pbs            12.682677
subtraction    11.605064
seek           11.605064
dtype: float64

### Which channels are best represented?

In [569]:
df[df.title.str.lower().str.contains('squad')].title

263       Kid Engineer: Uplift Wheelchair | Design Squad
346         Kid Engineer: Compost Tumbler | Design Squad
359        Kid Engineer: School Solutions | Design Squad
381      Kid Engineer: Fashion Sun Shield | Design Squad
436    How to Stay Away from Germs! | Odd Squad | PBS...
116         Kid Engineer: 3D-Printed Hand | Design Squad
209    Liverpool squad make heartwarming 2019 Christm...
330       How to Wash Your Hands! | Odd Squad | PBS KIDS
425    Odd Squad FULL EPISODE | Soundcheck / Double T...
98     ODD SQUAD | Training Video #306: How to Repair...
291         Kid Engineer: Recycling Nudge | Design Squad
104    Odd Squad FULL EPISODE | The Void / Into the O...
129    Kid Engineer: Upcycled Art Station | Design Squad
359    Rock Climbing for Kids with Disabilities | Des...
394             Kid Engineer: Lunch Table | Design Squad
198    Kid Engineer: Seed Launching Backpack | Design...
271       Kid Engineer: Underwater R.O.V. | Design Squad
465    Kid Engineer: Growing Pl

In [537]:
df[df.madeForKids > 0].channelTitle.value_counts().head(20)

It's AumSum Time                               174
PBS KIDS                                       134
English Singsing                                48
Wow Kidz                                        48
YouTube Movies                                  36
Free School                                     35
Wolfoo Family                                   32
STEVE AND MAGGIE                                28
Crash Course Kids                               27
SciShow Kids                                    24
Lotty Learns                                    24
Dodo Kids                                       23
123ABCtv                                        23
Michaels Stores                                 20
Kevin David's marketer                          18
The Singing Walrus - English Songs For Kids     18
Yo Kidz                                         16
Design Squad Global                             15
Bi Bi Kids                                      15
Brain Candy TV                 

### Which search condition has the best hit rate for 'madeForKids' content?

In [544]:
df.groupby(['topicId'])[['madeForKids']].apply(lambda x: x.sum()/x.count()).sort_values('madeForKids', ascending=False)

,madeForKids
topicId,
/m/02vxn,0.905797
02jjt,0.611111
01k8wb,0.570934
/m/03glg,0.514423
/m/02jjt,0.450704
/m/0f2f9,0.440678
/m/01k8wb,0.411894
/m/09kqc,0.373626
/m/019_rr,0.286932


In [545]:
df.groupby(['sortBy'])[['madeForKids']].apply(lambda x: x.sum()/x.count()).sort_values('madeForKids', ascending=False)

,madeForKids
sortBy,
relevance,0.532057
viewCount,0.432292
rating,0.398148
date,0.380339


In [546]:
df.groupby(['topicId', 'sortBy'])[['madeForKids']].apply(lambda x: x.sum()/x.count()).sort_values('madeForKids', ascending=False)

madeForKids
topicId    sortBy                
/m/02vxn   date          0.905797
02jjt      relevance     0.611111
01k8wb     relevance     0.570934
/m/03glg   date          0.514423
/m/02jjt   rating        0.500000
/m/01k8wb  viewCount     0.454268
/m/0f2f9   date          0.440678
/m/02jjt   viewCount     0.440000
           date          0.428571
/m/09kqc   date          0.373626
/m/0bzvm2  rating        0.357143
/m/01k8wb  rating        0.325000
/m/0bzvm2  date          0.295146
/m/01k8wb  date          0.290698
/m/019_rr  date          0.286932
/m/0bzvm2  relevance     0.196532
           viewCount     0.193548
/m/0403l3g date          0.056872

## Step #2: Download Transcripts

In [566]:
ddf = dd.read_parquet('data/yt_info/details*').drop_duplicates('videoId')
just_for_kids = ddf[ddf.madeForKids == 1].compute()
subtitle_path = 'data/yt_captions'
problem_vids = ['YSBDOY5ereM', 'iXfUoXTQBFc', 'vuYE0KDqZt0', 'kGKLOk7C7D0', 'lDCoaZNABug', 'LKefHZ6Ll38', 'tRazen6PWgk', 
                'sO3gL3Xfj-Y', 'YJs4xKMSey4', 'HbN-wISvLgc', '-P27nmY2RYQ', 'V5MzpyltQAs', 'kbr-u2iSNeg', 'whH9oDAusFw',
                'fpsuMQbmlfM', 'STcHx7Ze-vo', '5WGJXK2Wg2E', 'TlT0VXxSILM', 'jgDYOChgRG8', '4aYrJ0KfGxc', 'FIKoPWNldXc',
                '0B70aabIOOA', '_ZHcVzvGcwA', 'DuA8lzug7c0', 'CfwyTltnBB4', 'gGiJ6xJsMp4', '7FFMlV0iUyw', 'SrNiyDQPtjQ',
                'jW5UN1Vx6Kc', 'sD5A2SaTleA', 'YSxB0HOjgQ8', 'HRi5UKXRo-w', 'gbSGh0aafWY', 'pommGQO4MLU', 'ES6SZvYnCkw',
                'yToFwYlxit0'] # Copy from subtitle scraping errors below
already_completed = [f[:-5] for f in os.listdir(subtitle_path) if f.endswith('.json')]
already_completed += problem_vids
print(len(just_for_kids), len(already_completed))
just_for_kids = just_for_kids[~just_for_kids.videoId.isin(already_completed)]
just_for_kids.head(2)

1881 2058


,videoId,title,description,publishedAt,channelTitle,q,topicId,sortBy,duration,caption,relevantTopicIds,topicCategories,madeForKids


In [564]:
err_vids = []
for i, (ind, vid) in enumerate(just_for_kids.iterrows()):
    print(i, vid.title)
    try:
        yt.fetch_and_save_transcript(vid.videoId, subtitle_path)
    except KeyboardInterrupt:
        raise
    except youtube_transcript_api.TranscriptsDisabled:
        print('Err with {}'.format(vid.videoId))
        err_vids.append(vid.videoId)
        continue
    time.sleep(np.random.randint(100, 1000)/100)

0 Little Pim: Let&#39;s Play! - Arabic for Kids
Err with YJs4xKMSey4
1 Little Pim: Get up and Go! - Arabic for Kids
Err with HbN-wISvLgc
2 Little Pim: Let&#39;s Play! - Korean for Kids
Err with -P27nmY2RYQ
3 Little Pim: Get up and Go! - Korean for Kids
Err with V5MzpyltQAs
4 Little Pim: Let&#39;s Eat! - Korean for Kids
Err with kbr-u2iSNeg
5 Little Pim: At Home - Italian for Kids
Err with whH9oDAusFw
6 Little Pim: Let&#39;s Count - Italian for Kids
Err with fpsuMQbmlfM
7 Little Pim: At Home - German for Kids
Err with STcHx7Ze-vo
8 Little Pim: Let&#39;s Count - English for Kids
Err with 5WGJXK2Wg2E
9 Little Pim: At Home - English for Kids
Err with TlT0VXxSILM
10 Little Pim: Let&#39;s Eat! - Italian for Kids
Err with jgDYOChgRG8
11 Little Pim: Get Up and Go! - Italian for Kids
Err with 4aYrJ0KfGxc
12 Little Pim: Let&#39;s Eat! - English for Kids
Err with FIKoPWNldXc
13 Little Pim: Let&#39;s Play! - Italian for Kids
Err with 0B70aabIOOA
14 Little Pim: Let&#39;s Eat! - Chinese for Kids
Err

## Step 3: Parse Dataset

See [YTCaptionParsing.ipynb](YTCaptionParsing.ipynb)